In [ ]:
@patch("rss_collector.feed_parser.parse_feed")
def test_process_feeds_entry_validation(mock_parse_feed, test_feed_urls, mock_proxy_settings):
    """
    Test the process_feeds function to validate feed_data.entries handling.
    """
    # Mock proxy settings
    with patch("rss_collector.utils.get_proxy_settings", return_value=mock_proxy_settings):
        # Mock valid and invalid feed_data
        valid_feed_data = MagicMock()
        valid_feed_data.entries = [
            MagicMock(title="Valid Article", link="http://example.com/article", published="2024-12-01T00:00:00Z")
        ]

        invalid_feed_data = MagicMock()
        invalid_feed_data.entries = []  # Simulate empty entries

        mock_parse_feed.side_effect = [valid_feed_data, invalid_feed_data]  # First feed valid, second invalid

        process_feeds(test_feed_urls)

        # Verify that the valid feed is processed
        mock_parse_feed.assert_called()
        # Check that metadata for valid feed is saved
        saved_files = os.listdir("rss_metadata")
        assert len(saved_files) == 1  # Only the valid feed should produce metadata
        with open(os.path.join("rss_metadata", saved_files[0]), "r") as f:
            metadata = json.load(f)
        assert len(metadata["articles"]) == 1  # Only one article from valid feed
